# Quantum ALU

In [3]:
from qiskit import *
import numpy as np
from math import pi, sqrt
import random

In [4]:
backend = Aer.get_backend('statevector_simulator')
def getBloch(quantumCircuit):
    return visualization.plot_bloch_multivector(execute(quantumCircuit, backend).result().get_statevector())

In [5]:
def stateGenerator(state):
    state = state[::-1]
    current = int(state, 2)
#     print('Current state: ', current)
    newState = [0 for i in range(0, 2**(len(state)))]
    newState[current] = 1
#     print('Init state array: ',newState)
    return newState

In [6]:
def getMeasure(quantumCircuit, n):
    measures = []
    for i in range(0,n):
        outcome = execute(quantumCircuit, backend).result()
        hist = outcome.get_counts()
        for i in hist.keys():
            measures.append(i)
        return measures

In [7]:
def makeCirc(n,op, inp):
    initState = op + inp
    print(initState)
    qr = QuantumRegister(n)
    cr = ClassicalRegister(n)
    circ = QuantumCircuit(qr, cr)
    initializedState = stateGenerator(initState)
    circ.initialize(initializedState, qr)
    return circ

In [8]:
opLUT = {'+': '11000',
         '-': '11100',
         'xor': '10000',
         'xnor': '10100',
         'nop': '00000',
         '+1': '11010',
         '-1': '11111',
         'neg': '00100',
        }
# Look up table - FPGA

In [9]:
#ALU

In [14]:
def performOp(inp1, inp2, op):
    circuit = makeCirc(7, opLUT[op], str(inp1) + str(inp2))
    circuit.csx(0,6)
    circuit.csx(5,6)
    circuit.cx(0,5)
    circuit.crx(-pi/2, 5, 6)
    circuit.cx(0,5)
    circuit.csx(1, 6)
    circuit.csx(3, 6)
    circuit.cx(1, 3)
    circuit.crx(-pi/2, 3, 6)
    circuit.cx(2, 6)
    circuit.cx(4, 6)
    circuit.measure(6,6)
    return getMeasure(circuit, 1)[0][0]

In [20]:
performOp(1, 1, '+')

1100011


'0'

In [23]:
performOp(0, 1, 'xor')

1000001


'1'

In [25]:
performOp(1, 0, '+1')

1101010


'0'